In [1]:
import numpy as np
import torch
import sys
sys.path.append('../')
from voting_games.werewolf_env_v0 import raw_env
import random
from tqdm import tqdm

/root/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Done


In [3]:
import enum

In [4]:
class Phase(enum.IntEnum):
    ACCUSATION = 0
    VOTING = 1
    NIGHT = 2

In [13]:
def get_game_phase():
    phase = Phase(0)
    total_phases = len(Phase)
    while True:
        yield phase
        phase = Phase(phase+1)

In [14]:
gen = get_game_phase()

In [18]:
next(gen)

ValueError: 3 is not a valid Phase

In [2]:
from tianshou.data import Collector
from tianshou.env import DummyVectorEnv, PettingZooEnv
from tianshou.policy import MultiAgentPolicyManager, RandomPolicy

In [6]:
# env = raw_env(num_agents=10, werewolves=2)
env = raw_env(num_agents=5, werewolves=1)
env.reset()

In [7]:
def flat_obs(observation):
    return  [observation['day']] + \
            [observation['phase']] + \
            [int(status) for status in observation['player_status']] + \
            [role for role in observation['roles']] + \
            [vote for vote in observation['votes']]

In [8]:
env = PettingZooEnv(env)

In [9]:
policies = MultiAgentPolicyManager([RandomPolicy(), RandomPolicy(), RandomPolicy(), RandomPolicy(), RandomPolicy()], env)

In [10]:
env = DummyVectorEnv([lambda: env])

In [11]:
collector = Collector(policies, env)

In [14]:
result = collector.collect(n_episode=20)

In [15]:
result

{'n/ep': 20,
 'n/st': 352,
 'rews': array([[ 24., -31.,  23.,  23.,  23.],
        [ 22., -33.,  17.,  18.,  20.],
        [-29., -32.,  21., -33., -32.],
        [-32., -33.,  20., -32., -29.],
        [ 24.,  23.,  24.,  23., -31.],
        [ 18.,  22.,  21.,  17., -33.],
        [ 17.,  21., -33.,  18.,  20.],
        [-33.,  21., -32., -34., -29.],
        [-31.,  24.,  24.,  23.,  24.],
        [-31.,  19.,  24.,  23.,  23.],
        [-33., -32.,  21., -29., -33.],
        [ 24., -32.,  23.,  19.,  23.],
        [ 21.,  21.,  18.,  17., -33.],
        [ 19., -32.,  23.,  23.,  23.],
        [ 17.,  22.,  21., -33.,  18.],
        [ 21., -28., -33., -33., -32.],
        [ 18.,  18.,  21.,  20., -34.],
        [-33.,  21., -28., -33., -32.],
        [-32.,  20., -32., -28., -34.],
        [ 21., -33.,  17.,  18.,  20.]]),
 'lens': array([10, 20, 20, 20, 10, 20, 20, 20, 10, 14, 20, 14, 20, 14, 20, 20, 20,
        20, 20, 20]),
 'idxs': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,